In [53]:
# Initialization & Imports and such..
import json
import pyautogui
import pytesseract
import cv2
import os
import numpy as np
from datetime import datetime
from time import sleep
from mss import mss
from PIL import Image
# Uncomment the below if having issues running PyTesseract because it's not found.
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe"

# Run pip install strsim - This is the pythonn-strinng-similarity library

# Import all card names
# Pull json back thru json file created above.
with open('all_cards.json', 'rb') as f:
    all_cards = json.load(f) # json/dict containing total_cards and list of list containing card name and hyperlink for card.
# Let's create code to get all card_name's in a list to for our use case...
just_names = []
for card in all_cards['all_cards']:
    just_names.append(card[0])
len(just_names) # Should be total number of cards in EternalWarcry db at any time (currently 1754).

1754

In [54]:
'''
This Takes an input of OCR'ed text, and outputs the most similar text to it.
'''
def find_card(tess_text):
    ## TODO: MAKE IT ONLY SEARCH THE SUBSET OF CARDS IN THE DECK WE ARE PLAYING :) 
    from similarity.normalized_levenshtein import NormalizedLevenshtein
    normalized_levenshtein = NormalizedLevenshtein()

    prob_card = ''
    prob_similarity = float(0.0)

    for card in just_names:
        current_prob = normalized_levenshtein.similarity(card_name, card)
        #print(card + 'Similarity to: ' + card_name + ' is ' + str(current_prob))
        if current_prob > prob_similarity:
            prob_similarity = current_prob
            prob_card = card

    print('Winner is: ' + prob_card + '. Original tesseract_text= ' + card_name)
    return prob_card

'''
Methods found online to try different thresholding + binarization methods,
and settling on 2/3 being the best. These are use in the third part of process/block.
'''

def apply_threshold(img, argument):
    switcher = {
        1: cv2.threshold(cv2.GaussianBlur(img, (9, 9), 0), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1],
        2: cv2.threshold(cv2.GaussianBlur(img, (7, 7), 0), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1],
        3: cv2.threshold(cv2.GaussianBlur(img, (5, 5), 0), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1],
        4: cv2.threshold(cv2.medianBlur(img, 5), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1],
        5: cv2.threshold(cv2.medianBlur(img, 3), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1],
        6: cv2.adaptiveThreshold(cv2.GaussianBlur(img, (5, 5), 0), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2),
        7: cv2.adaptiveThreshold(cv2.medianBlur(img, 3), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2),
    }
    return switcher.get(argument, "Invalid method")

def get_string(img_path, method):
    # Read image using opencv
    #img = cv2.imread(img_path)# Unncomment if want input to be FILE nnot cv2.im object
    img = img_path

    # Convert to gray
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)
    
    # invert threshold
    img = cv2.bitwise_not(img)
    
    # Apply threshold to get image with only black and white
    img = apply_threshold(img, method)
    
    save_path = 'screenshots/post_image_tess.jpg'
    cv2.imwrite(save_path, img)

    # Recognize text with tesseract for python
    result = pytesseract.image_to_string(img, lang="eng")#, config="-c tessedit_char_whitelist=0123456789abcdefghijklmnopqrstuvwxyz")
    return result

In [ ]:
'''
REQUIREMENT: ETERNAL RUNNING FULLSCREEN/BORDERLESS ON 1920X1080 monitor, and monitor_num=2 if on second monitor.
(monitor_num=1 if not). Grabs an image of monitor (1920x1080) every 3 seconds
'''
# Set which monitor you want (1|2) it to continually capture and save to...
monitor_num = 2

with mss() as sct:
    while 'capturing':
        filename = 'screenshots/' + datetime.today().strftime("%Y%m%d_%H%M%S.jpg")
        sct.shot(mon=monitor_num, output=filename)
        sleep(1)

In [71]:
'''
Crops image to encompass the region where the CARD_NAMES will be located.
Format for cropping = #img[y:y+h, x:x+w]
'''
# Change this to filename eventually to match
screenshot = "screenshots/7.jpg"
#############################################

img = cv2.imread(screenshot)
crop_img = img[700:792, 385:1500] # OG FRAME = img[663:788, 385:1500]
#cv2.imwrite(screenshot, crop_img)

'''
Get Card name then compare against imported collection.
'''

# Use 2 or 3 for deciphering. There is also 'Cronos' lang if running into issue
card_name = get_string(crop_img, 2) #input was originally FILE, inputting img from above. (screenshot, 2)

if not card_name:# | 'cant find similarity past threshold'
    card_name = get_string(crop_img, 3)
elif not card_name:
    raise ValueError('get_string was unable to to OCR a string. Try all methods 1-7, or re-crop.')
#print(card_name)

# LAST STEP: Similarity comparisonn
true_name = find_card(card_name)
print('\n\n\n' + true_name)

Winner is: Vara's Favor. Original tesseract_text= 4 Vara's Favor 1

_- __
' . e



Vara's Favor


In [35]:
# Success! Break for today.... Well done Lane :))))))        

# Test stuff below, above we keep./s

In [ ]:
import mss
import mss.tools


with mss.mss() as sct:
    # Get information of monitor 2
    monitor_number = 2
    mon = sct.monitors[monitor_number]

    # The screen part to capture {X=3077,Y=1034,Width=1006,Height=304}
    monitor = {
        "top": 517,  # 100px from the top mon["top"] + 
        "left": 587,  # 100px from the left mon["left"] + 
        "width": 914,
        "height": 776,
        "mon": monitor_number,
    }
    output = "sct-mon{mon}_{top}x{left}_{width}x{height}.png".format(**monitor)

    # Grab the data
    sct_img = sct.grab(monitor)

    # Save to the picture file
    mss.tools.to_png(sct_img.rgb, sct_img.size, output=output)
    print(output)

Fire Sigil


In [13]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
img = cv.imread('screenshots/test5.png',0)
img = cv.medianBlur(img,5)
#ret,th1 = cv.threshold(img,127,255,cv.THRESH_BINARY)
#th2 = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_MEAN_C,\
#            cv.THRESH_BINARY,11,2)
th3 = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv.THRESH_BINARY,11,2)
titles = ['Original Image', 'Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [th3]#[img, th1, th2, th3]
for i in range(4):
    plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()

IndexError: list index out of range

In [15]:
import cv2
import numpy as np
import pytesseract
from PIL import Image


def get_string(img_path):
    # Read image with opencv
    img = cv2.imread(img_path)

    # Convert to gray
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    


    # Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)

    # Write image after removed noise
    cv2.imwrite("removed_noise.png", img)

    #  Apply threshold to get image with only black and white
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, -15)
    
    # invert threshold
    img = cv2.bitwise_not(img)

    # Write the image after applying opencv...
    cv2.imwrite("thresh.png", img)

    # Recognize text with pytesseract
    result = pytesseract.image_to_string(Image.open("thresh.png"), lang='eng')#, lang='eng',
                        #config='--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789')

    return result
print(get_string('screenshots/test5.png'))

-

(*~ Cipt

k‘ hus' Might


In [16]:
import pytesseract
from PIL import Image
text = pytesseract.image_to_string(Image.open("screenshots/test.png"), lang='eng',
                        config='--oem 3 -c tessedit_char_whitelist=0123456789')

print(text)

FileNotFoundError: [Errno 2] No such file or directory: 'screenshots/test.png'

In [17]:
# This block takes image and outputs text.
# NOTE: youll need to pip Pillow and pytesseract, and then install the windows tesseract along with mapping the path here:
# https://github.com/tesseract-ocr/tesseract/wiki/4.0-with-LSTM#400-alpha-for-windows
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe" # paths correctly

try:  
    from PIL import Image
except ImportError:  
    import Image
import pytesseract

def ocr_core(filename):  
    """
    This function will handle the core OCR processing of images.
    """
    im = Image.open(filename)
    text = pytesseract.image_to_string(im, lang="eng")  # We'll use Pillow's Image class to open the image and pytesseract to detect the string in the image
    return text

print(ocr_core('screenshots/test5.png'))

(*~ Caiphus Might 1k
